In [34]:
import json
import random
import pandas as pd
import sqlite3

# feature_table(ft) をDataFrame形式で読み込み
def load_feature_table(dbpath, tablename="feature_table"):
    conn=sqlite3.connect(dbpath)
    c = conn.cursor()
    ft = pd.read_sql('SELECT * FROM ' + tablename, conn)
    return ft

# feature_table_indexes(ftis) を初期化
def init_feature_table_indexes(feature_table):
    ft = feature_table
    ftis = {} # feature_table_indexes
    ftis["train"],ftis["trained"], ftis["query"], ftis["selected_query"] = [], [], [], []
    labels = sorted(ft["label"].unique())
    ft_labelby = ft.groupby("label")
    for label in labels:
        df = ft_labelby.get_group(label)
        queries = df["index"].values.tolist()
        query = random.sample(queries, 1)[0]
        ftis["query"].append(query)
    return ftis

# feature_table_indexes(ftis) をjson形式で保存
def save_feature_table_indexes(ftis, savepath):
    with open(savepath, "w") as f:
        json.dump(ftis, f, indent=4)
        
# feature_table_indexes(ftis) を辞書形式で読み込み
def load_feature_table_indexes(ftis_path):
    with open(ftis_path, "r") as f:
        ftis = json.load(f)
    return ftis

In [49]:
import json
import faiss
import numpy as np

class TrainDataSelector:
    
    def __init__(self, feature_table, feature_table_indexes):
        self.ft = self.__drop_trained_data(feature_table, feature_table_indexes)
        self.ftis = feature_table_indexes
        self.labels = sorted(feature_table["label"].unique())
        self.faiss_indexes = {} # ラベルごとのフィーチャ全体のfaissインデックス
              
    def __drop_trained_data(self, feature_table, feature_table_indexes):
        ft = feature_table.drop(index=feature_table_indexes["trained"])
        ft = ft.reset_index(drop=True)
        return ft
    
    # フィーチャを検索するための,フィーチャ全体のfaissインデックスを作成
    def make_faiss_indexes(self):
        ft_labelby = self.ft.groupby("label")
        for label in self.labels:
            features = []
            df = ft_labelby.get_group(label)
            for feature in df["feature"]:
                features.append(json.loads(feature))
            features = np.array(features).astype("float32")
            dim = len(features[0])
            index = faiss.IndexFlatL2(dim)
            index.add(features)
            self.faiss_indexes[label] = index

In [ ]:
dbpath = "./assets/features_v1.db"
feature_table = load_feature_table(dbpath)

In [35]:
tables_labelby = []
labels = feature_table["label"].unique()
df = feature_table.groupby("label")
for label in labels:
    df_labelby = df.get_group(label)
    df_labelby = df_labelby.reset_index(drop=True)
    tables_labelby.append(df_labelby)

In [ ]:
for table in tables_labelby:
    features = []
    for feature in table["feature"]:
        feature = json.loads(feature)
        features.append(feature)

In [ ]:
for table in self.tables_labelby:
    features = []
    for feature in table["feature"]:
        feature = json.loads(feature)
        features.append(feature)
    features = np.array(features).astype("float32")
    dim = len(features[0])
    index = faiss.IndexFlatL2(dim)
    index.add(features)
    self.feature_indexes.append(index)

In [48]:
a = {}
a[0] = 1
a

{0: 1}